# Backtest a strategy based on a high & low price

1. Go long when the stock closes above the last 20 day’s high price.
2. Square off the long position when the stock goes below the last 20 day’s low price.
3. Optional: Optimise the strategy by adjusting the number of periods. You can choose to have different number of periods for entering the long and exiting the long.

Import the required libraries

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import pyfolio
import pandas_datareader.data as web
import talib as ta


C:\Users\ragav\anaconda3\lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [2]:
# Define the backtesting period and here it is 1 year
end = pd.datetime.now().date()
start = end - pd.Timedelta(days=1*252)

<ipython-input-2-02cf5970d37d>:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  end = pd.datetime.now().date()


In [3]:
#Create a backtesting class
class backtest_High_Low:
    
    #Create instance attributes/methods
    def __init__(self,ticker,start,end,ndays_high,ndays_low):
        
        
        self.ticker = ticker
        self.start = start
        self.end = end
        self.ndays_high = ndays_high
        self.ndays_low = ndays_low
        
        self.fetch_data()
        self.indicators()
        self.process()
        self.signals()
        self.positions()
        self.returns()
    
    # Fetch the data from yahoo finance
    def fetch_data(self):
        self.df = yf.download(self.ticker,self.start,self.end)
    
    #Compute the indicators of number of days low & high
    def indicators(self):
        self.df['ndays_high'] = self.df['Adj Close'].rolling(window = self.ndays_high).max().shift(1)
        self.df['ndays_low'] = self.df['Adj Close'].rolling(window = self.ndays_low).min().shift(1)
    
    #Clean the data by dropping NaN values
    def process(self):
        self.df.dropna(inplace=True,axis=0)
    
    # Generate the entry and exit signals
    def signals(self):
        self.df['signals'] = np.where(self.df['Adj Close'] > self.df['ndays_high'],1,0)
        self.df['signals'] = np.where(self.df['Adj Close'] < self.df['ndays_low'], -1, self.df['signals'])
        
        self.df['signals'] = self.df['signals'].replace(to_replace=0,method='ffill')
        self.df['signals'] = self.df['signals'].replace(to_replace=-1,value=0)
    
    # Mark the positions taken
    def positions(self):
        self.df['positions'] = self.df['signals'].shift(1)
    
    # Generate the buy & hold and strategy returns
    def returns(self):
        
        # BnH returns are the returns generated over the mentioned period if bought and hold it over
        self.df['BnH Returns'] = self.df['Adj Close'].pct_change()
        # Strategy returns are the returns generated after implementing this strategy
        self.df['Strategy_Returns'] = self.df['BnH Returns']*self.df['positions']
        print('Strategy Returns :' ,self.df['Strategy_Returns'].cumsum()[-1])
        return self.df['Strategy_Returns'].cumsum()[-1]
        

 Try this strategy to see the returns generated for different indices with different range of low and high

In [4]:
indice_list=['^NSEI','SPY','^HSI','^CNXIT','^NSEBANK','^HSI','^GSPC']

high_range=[]
low_range=[]
indices =[]
Returns=[]

for i in range(8,22,2):
    for j in range(8,22,2):
        for k in indice_list:
            a = backtest_High_Low(k,start,end,i,j)
            high_range.append(i)
            low_range.append(j)
            Returns.append(a.returns())
            indices.append(k)

[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.0585122628266358
Strategy Returns : 0.0585122628266358
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.003208504673833046
Strategy Returns : 0.003208504673833046
[*********************100%***********************]  1 of 1 completed
Strategy Returns : -0.10675537759134968
Strategy Returns : -0.10675537759134968
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.22092278938206888
Strategy Returns : 0.22092278938206888
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.07394250011725245
Strategy Returns : 0.07394250011725245
[*********************100%***********************]  1 of 1 completed
Strategy Returns : -0.10675537759134968
Strategy Returns : -0.10675537759134968
[*********************100%***********************]  1 of 1 completed
Strategy Returns : -0.004073060858400979
Stra

[*********************100%***********************]  1 of 1 completed
Strategy Returns : -0.008647715879819162
Strategy Returns : -0.008647715879819162
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.07988327400391992
Strategy Returns : 0.07988327400391992
[*********************100%***********************]  1 of 1 completed
Strategy Returns : -0.004299845283769699
Strategy Returns : -0.004299845283769699
[*********************100%***********************]  1 of 1 completed
Strategy Returns : -0.06765709727516778
Strategy Returns : -0.06765709727516778
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.18125891748548384
Strategy Returns : 0.18125891748548384
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.0913413041228538
Strategy Returns : 0.0913413041228538
[*********************100%***********************]  1 of 1 completed
Strategy Returns : -0.06765709727516778
S

[*********************100%***********************]  1 of 1 completed
Strategy Returns : -0.09609006255189234
Strategy Returns : -0.09609006255189234
[*********************100%***********************]  1 of 1 completed
Strategy Returns : -0.028722121507362197
Strategy Returns : -0.028722121507362197
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.0955905452615391
Strategy Returns : 0.0955905452615391
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.028435573252051927
Strategy Returns : 0.028435573252051927
[*********************100%***********************]  1 of 1 completed
Strategy Returns : -0.1274457618614928
Strategy Returns : -0.1274457618614928
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.12695410197979817
Strategy Returns : 0.12695410197979817
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.02315452533826934
Stra

[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.016905164340537904
Strategy Returns : 0.016905164340537904
[*********************100%***********************]  1 of 1 completed
Strategy Returns : -0.1292912558063939
Strategy Returns : -0.1292912558063939
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.009977603699847903
Strategy Returns : 0.009977603699847903
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.06954371368542578
Strategy Returns : 0.06954371368542578
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.011701167371456433
Strategy Returns : 0.011701167371456433
[*********************100%***********************]  1 of 1 completed
Strategy Returns : -0.1383270400540506
Strategy Returns : -0.1383270400540506
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.1332041367215091
Strat

[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.123868560193088
Strategy Returns : 0.123868560193088
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.05098511891791013
Strategy Returns : 0.05098511891791013
[*********************100%***********************]  1 of 1 completed
Strategy Returns : -0.06399986504302757
Strategy Returns : -0.06399986504302757
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.02338837752927847
Strategy Returns : 0.02338837752927847
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.0372370826038545
Strategy Returns : 0.0372370826038545
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.010427524715955316
Strategy Returns : 0.010427524715955316
[*********************100%***********************]  1 of 1 completed
Strategy Returns : -0.06399986504302757
Strategy Re

[*********************100%***********************]  1 of 1 completed
Strategy Returns : -0.024621880983546007
Strategy Returns : -0.024621880983546007
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.16659608933802172
Strategy Returns : 0.16659608933802172
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.05098511891791013
Strategy Returns : 0.05098511891791013
[*********************100%***********************]  1 of 1 completed
Strategy Returns : -0.024621880983546007
Strategy Returns : -0.024621880983546007
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.0036323961852495756
Strategy Returns : 0.0036323961852495756
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.036438828178719906
Strategy Returns : 0.036438828178719906
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.04523638826936

[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.04523638826936238
Strategy Returns : 0.04523638826936238
[*********************100%***********************]  1 of 1 completed
Strategy Returns : -0.07449740116578851
Strategy Returns : -0.07449740116578851
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.19455543222529892
Strategy Returns : 0.19455543222529892
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.03911909593801999
Strategy Returns : 0.03911909593801999
[*********************100%***********************]  1 of 1 completed
Strategy Returns : -0.07449740116578851
Strategy Returns : -0.07449740116578851
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.03919894759309139
Strategy Returns : 0.03919894759309139
[*********************100%***********************]  1 of 1 completed
Strategy Returns : 0.03424058648578654
Strate

In [7]:
# Convert the results into dataframe and sort the data by strategy returns 
Results = pd.DataFrame({'Indices':indices,'High':high_range,'Low':low_range,'Returns':Returns})
Results.sort_values(by='Returns',inplace=True,ascending=False)

In [8]:
Results.head(10)

,Indices,High,Low,Returns
87,^CNXIT,10,18,0.236127
38,^CNXIT,8,18,0.236127
3,^CNXIT,8,8,0.220923
136,^CNXIT,12,18,0.220645
185,^CNXIT,14,18,0.220645
234,^CNXIT,16,18,0.216984
283,^CNXIT,18,18,0.211622
45,^CNXIT,8,20,0.203901
94,^CNXIT,10,20,0.203901
10,^CNXIT,8,10,0.202408
